In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders.one_hot import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.metrics import classification_report

#### Importing the data

In [ ]:
# Importing the train data
train = pd.read_csv(r'../input/spaceship-titanic/train.csv',sep=',')
train.head()

In [ ]:
# Importing the test data
test = pd.read_csv(r'../input/spaceship-titanic/test.csv',sep=',')
test.head()

#### Cleaning the Data

In [ ]:
train.isnull().sum()

In [ ]:
train.fillna(0,inplace = True)

In the 'Cabin' axis there are 3 'hidden' variables. Let's split them to perform the transformation to numeric features later.

In [ ]:
train[['Deck','Num','Side']]= train['Cabin'].str.split(pat='/',expand=True)

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.fillna(0,inplace=True)

In [ ]:
train['Num'] = train['Num'].astype(int)

Making a One Hot Encoder with categorical features

In [ ]:
one_hot_encoder = OneHotEncoder(cols=['HomePlanet','CryoSleep','Destination','VIP','Deck','Side'])

In [ ]:
train = one_hot_encoder.fit_transform(train)

Excluding the categorical data

In [ ]:
train.drop(['PassengerId','Name','Cabin'],inplace = True,axis=1)

#### Spliting the data

Initially, we will split the data into training and testing. For the data below, I'll split it as 80/20.

In [ ]:
y = train['Transported']
X = train.drop(['Transported'],axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#### Random Forest Classifier

First, we'll use Random Forest to see how it will perform.

In [ ]:
clf = RandomForestClassifier(max_depth=2,random_state=0)

In [ ]:
clf.fit(X_train, y_train)

Random Forest performed well, let's test ADABoost now to see how it performs.

In [ ]:
predict_RF = clf.predict(X_val)

accuracy_score(y_val, predict_RF)

In [ ]:
cm_randomforest = confusion_matrix(y_val, predict_RF)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_randomforest,
                                display_labels=clf.classes_)
disp_rf.plot()
plt.show()

In [ ]:
print(classification_report(y_val, predict_RF))

#### ADABoosting Classifier

In [ ]:
regr = AdaBoostClassifier(random_state=42, n_estimators=100)

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
predict_ADA = regr.predict(X_val)

In [ ]:
accuracy_score(y_val, predict_ADA)

In [ ]:
cm_ada = confusion_matrix(y_val, predict_ADA)
disp_ada = ConfusionMatrixDisplay(confusion_matrix=cm_ada,
                                display_labels=regr.classes_)
disp_ada.plot()
plt.show()

In [ ]:
print(classification_report(y_val, predict_ADA))

Finally, let's test Gradient Boosting to see if it will be more accurate.

#### Gradient Boosting Classifier

In [ ]:
GBC = GradientBoostingClassifier(n_estimators = 100,random_state=42,max_depth=2,learning_rate=1.0,max_features=2)


In [ ]:
GBC.fit(X_train, y_train)

In [ ]:
predict_gbc = GBC.predict(X_val)

In [ ]:
accuracy_score(y_val, predict_gbc)

In [ ]:
cm_gbc = confusion_matrix(y_val, predict_gbc)
disp_gbc = ConfusionMatrixDisplay(confusion_matrix=cm_gbc,
                                display_labels=GBC.classes_)
disp_gbc.plot()
plt.show()

In [ ]:
print(classification_report(y_val, predict_gbc))

#### Conclusion


Finally, for the data in question we had a better performance using Gradient Boosting. However, we also obtained satisfactory results with Random Forest and ADABoosting.

In [ ]:
print('Random Forest:',accuracy_score(y_val, predict_RF))
print('ADABoost:',accuracy_score(y_val, predict_ADA))
print('Gradient Boosting Classifier:',accuracy_score(y_val, predict_gbc))